## get Cause efffect phrase from the sentence.

In [1]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

#'data/in/semeval/sem_CE_test_cleaned.csv'
#'data/in/RAG/ADE/ADE_CE_extract_TEST.csv')
#'data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv'


pm_DB = pd.read_csv('data/in/KNN_RAG/ADE_CE_extract_TEST.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]
instances = pm_DB['sentence'].to_list()
len(instances)

2271

In [2]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()
len(examples)

2365

In [3]:
semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

In [4]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [5]:
import groq
from groq import Groq
client = groq.Groq(api_key= "enter your API key here..")#sameer key

#gemma2-9b-it# llama3-8b-8192

In [6]:
def C_E_CP_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause": "", "effect": "", "causality_phrase": ""}.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
                   
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
                   
          ''' + fewshot_examples
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="gemma2-9b-it",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [7]:
def get_fewshotexamples_C_E_CP(fewshot_index):

    
    #fewshot_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    
    train_sentence = semeval_data['sentence'].to_list()
    train_causalityPhrase = semeval_data['causality_phrase'].to_list()

    fewshot_example_sentence = [train_sentence[i] for i in fewshot_index]
    fewshot_example_causalityPhrase = [train_causalityPhrase[i] for i in fewshot_index]


    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_example_causalityPhrase[i]} 
                    for i, sentence in enumerate(fewshot_example_sentence)}
    
    # Assuming json_data is already defined
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_example_sentence, fewshot_example_causalityPhrase

In [8]:
# semeval
filtered_rows = pm_DB

In [9]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        #print('ind :', ind)
        window_size = len(p)

        # Slide through input_sentence with a window size 
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('causality phrase: ',p, 'index is', ind)
                #print(f"Window '{window}' - LS: {score}")

    #print('fewshot_index', fewshot_index)
    fewshot_index = list(set(fewshot_index))
    #print('fewshot_index', fewshot_index)
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [10]:
import json
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
filtered_rows = pm_DB

cause_list = []
effect_list = []
causality_phrase_list = []
fewshotfound_inputsentence_index=[]
for i in range (len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    
    
    #KNN+RAG few shot examples augmentation.
    fewshot_index = get_fewshot_examples_index(input_sentence)############FEWSHOT
    
    fewshot_sentence_rag, fewshot_patterns_rag = get_fewshotexamples_C_E_CP(fewshot_index) #############fewshot
    #fewshot_examples =  get_randomfewshotexamples()##################random shot/zero shot
        
    #print('input_sentence : ', input_sentence)
    ###
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_sentence_knn = [examples[j] for j in most_similar_indices]
    fewshot_patterns_knn = [causality_phrase[p] for p in most_similar_indices]
    
    fewshot_sentence_comb = fewshot_sentence_rag+fewshot_sentence_knn
    fewshot_patterns_comb = fewshot_patterns_rag+fewshot_patterns_knn

    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_patterns_comb[i]} 
                    for i, sentence in enumerate(fewshot_sentence_comb)}
    
    fewshot_examples = json.dumps(fewshot_examples)   

    #print('fewshot_comb : ', fewshot_examples)
    
    #####end
    #####end
    # output generation
    try:
        out = C_E_CP_Identifier(input_sentence, fewshot_examples)

        print('out: ', out)

        json_out = json.loads(out)
        cause = json_out['cause']
        effect = json_out['effect']
        causality_phrases = json_out['causality_phrase']
        
    except TypeError as e:
        print('TypeError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
        
    except KeyError as e:
        print('KeyError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
    
    except groq.APIConnectionError as e:
        print('APIConnectionError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
        
    except groq.RateLimitError as e:
        print('RateLimitError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')       
        continue
        
    except groq.APIStatusError as e:
        print('APIStatusError')
        cause_list.append('')
        effect_list.append('')
        causality_phrase_list.append('')
        continue
    

    
    cause_list.append(cause)
    effect_list.append(effect)
    causality_phrase_list.append(causality_phrases)
    
    #print('input_sentence: ', input_sentence )
    #print('fewshot_examples: ', fewshot_examples )
    #print('cause: ', cause)
    #print('effect: ', effect)
    #print('causality_phrase: ', causality_phrases)
    
    
    print(i)
    #if i==0:
        #break;
    #print(causality_identifier)
    
    

out:  {"cause": "5-fluorouracil", "effect": "Squamous-cell carcinoma arising in a basal-cell epithelioma", "causality_phrase": "treated with"}
0
out:  {"cause": "long-term therapy with mitomycin C and 5-fluorouracil", "effect": "acute renal toxicity", "causality_phrase": "receiving"}
1
out:  {"cause": "budesonide", "effect": "contact dermatitis", "causality_phrase": "causing"}
2
out:  {"cause": "mannitol", "effect": "ARF", "causality_phrase": "induced"}
3
out:  {"cause": "chlorambucil was discontinued", "effect": "clinical AML", "causality_phrase": "the patient developed"}
4
out:  {"cause": "L-asparaginase therapy", "effect": "seizures and altered sensorium", "causality_phrase": "after"}
5
out:  {"cause": "a regimen of carbamazepine, lynestrenol and sodium valproate", "effect": "hepatopathy", "causality_phrase": "during"}
6
out:  {"cause": "anastrozole use", "effect": "Diagnosis of sclerosing glomerulonephritis", "causality_phrase": "during"}
7
out:  {"cause": "aspirin", "effect": "tra

out:  {"cause": "L-dopa treatment", "effect": "rapid mood cycling", "causality_phrase": "during"}
65
out:  {"cause": "the first dose of prazosin", "effect": "a sudden and severe drop in blood pressure", "causality_phrase": "after the administration of"}
66
out:  {"cause": "chlorpropamide", "effect": "Acute intravascular hemolysis", "causality_phrase": "was started on"}
67
out:  {"cause": "a higher dose than recommended was used, there was recent or concomitant treatment with mefloquine, there was pre-existing prolongation of the QT interval or the patient had a thiamine deficiency", "effect": "8 cardiac arrests, leading to 6 deaths", "causality_phrase": "when"}
68
out:  {"cause": "hypersensitivity reaction", "effect": "initial therapy with penicillin G and gentamicin was stopped and substituted with cefazolin", "causality_phrase": "Because of"}
69
out:  {"cause": "methylprednisolone pulse therapy", "effect": "Atrial fibrillation", "causality_phrase": "following"}
70
out:  {"cause": "es

out:  {"cause": "sulfadiazine", "effect": "crystalluria and renal failure", "causality_phrase": "induced"}
125
out:  {"cause": "isotretinoin (1 mg/kg/day)", "effect": "poor night vision and/or excessive glare sensitivity", "causality_phrase": "treated with"}
126
out:  {"cause": "calcineurin-induced pain syndrome (CIPS)", "effect": "a case of calcineurin-induced pain syndrome (CIPS)", "causality_phrase": "in a child undergoing"}
127
out:  {"cause": "treatment with olanzapine", "effect": "rapidly occurring hyperglycemia", "causality_phrase": "3 days after treatment with"}
128
out:  {"cause": "Magnesium tocolysis", "effect": "urinary calculus during pregnancy", "causality_phrase": "as the cause of"}
129
out:  {"cause": "use of sumatriptan", "effect": "cardiac arrest", "causality_phrase": "following"}
130
out:  {"cause": "amphotericin B administration", "effect": "Arrhythmias and cardiac arrest", "causality_phrase": "during"}
131
out:  {"cause": "clonidine", "effect": "bradycardia", "causa

out:  {"cause": "bevacizumab", "effect": "spontaneous nasal septal perforation", "causality_phrase": "after given"}
188
out:  {"cause": "anastrozole", "effect": "Renal injury", "causality_phrase": "due to"}
189
out:  {"cause": "extemporaneous topical minoxidil", "effect": "myocardial infarction", "causality_phrase": "had hypertension and arteriosclerotic disease and the other died of"}
190
out:  {"cause": "metoclopramide", "effect": "Reversible nonthrombocytopenic palpable purpura", "causality_phrase": "associated with"}
191
out:  {"cause": "miconazole", "effect": "Cardiorespiratory toxicity", "causality_phrase": "due to"}
192
out:  {"cause": "BCNU therapy", "effect": "fatal pulmonary fibrosis", "causality_phrase": "following"}
193
out:  {"cause": "tamoxifen for breast cancer", "effect": "Gastric tumor, endometrial carcinoma and cervical adenocarcinoma in situ", "causality_phrase": "after treatment with"}
194
out:  {"cause": "disulfiram", "effect": "hepatitis", "causality_phrase": "ind

out:  {"cause": "methotrexate-induced", "effect": "CNS abnormalities", "causality_phrase": "methotrexate-induced"}
248
out:  {"cause": "L-thyroxine therapy", "effect": "acute myocardial infarction", "causality_phrase": "due to"}
249
out:  {"cause": "a single dose of vincristine", "effect": "severe visual loss", "causality_phrase": "following"}
250
out:  {"cause": "penicillamine", "effect": "TTP in RA", "causality_phrase": "induced"}
251
out:  {"cause": "citalopram", "effect": "episodes of chest tightness and dizziness", "causality_phrase": "after starting"}
252
out:  {"cause": "DCF", "effect": "keratitis of moderate severity", "causality_phrase": "The ocular toxicity of"}
253
out:  {"cause": "glibenclamide therapy", "effect": "Pemphigus vulgaris", "causality_phrase": "precipitated by"}
254
out:  {"cause": "fluvoxamine-induced akathisia", "effect": "Beneficial effect of low-dose mianserin", "causality_phrase": "on"}
255
out:  {"cause": "", "effect": "", "causality_phrase": ""}
256
out: 

...

out:  {"cause": "bromocriptine therapy", "effect": "Morphea", "causality_phrase": "after"}
1965
out:  {"cause": "did not receive prochlorperazine", "effect": "retinal problems", "causality_phrase": "developed"}
1966
out:  {"cause": "carbamazepine administration", "effect": "syndrome of inappropriate antidiuresis", "causality_phrase": "as a much earlier side-effect of"}
1967
out:  {"cause": "Bisphosphonate", "effect": "osteonecrosis of the skull base", "causality_phrase": "-related"}
1968
out:  {"cause": "procainamide therapy", "effect": "clinical lupus syndrome with positive antinuclear antibody, positive lupus erythematosus (LE) cell preparation, and diffuse proliferative glomerulonephritis", "causality_phrase": "following"}
1969
out:  {"cause": "lithium therapy", "effect": "thyrotoxicosis with severe exophthalmos", "causality_phrase": "while on"}
1970
out:  {"cause": "carbamazepine and diphenylhydantoin", "effect": "Deposits of plasma proteins in the skin", "causality_phrase": "durin

out:  {"cause": "high doses of trazodone for treatment of an acute depression", "effect": "a movement disorder", "causality_phrase": "following"}
2028
out:  {"cause": "Cefoxitin therapy", "effect": "granulomatous hepatitis", "causality_phrase": "with associated"}
2029
out:  {"cause": "sunitinib", "effect": "acute interstitial nephritis", "causality_phrase": "associated with"}
2030
out:  {"cause": "systemic disease", "effect": "neurologic toxic effects", "causality_phrase": "associated with"}
2031
out:  {"cause": "nifedipine treatment", "effect": "Rupture of a cerebral aneurysm", "causality_phrase": "associated with"}
2032
out:  {"cause": "Methotrexate", "effect": "diffuse interstitial pulmonary fibrosis", "causality_phrase": "-induced"}
2033
out:  {"cause": "fludarabine treatment", "effect": "fatal intravascular autoimmune hemolytic anemia (AIHA)", "causality_phrase": "after"}
2034
out:  {"cause": "warfarin therapy in a patient with protein C deficiency", "effect": "Stuttering priapism

out:  {"cause": "acetazolamide", "effect": "Severe acidosis", "causality_phrase": "from"}
2093
out:  {"cause": "Naproxen", "effect": "tinnitus and transient hearing loss", "causality_phrase": "whose side effects include"}
2094
out:  {"cause": "two instillations of epirubicin", "effect": "significant morbidity related to primary and secondary perforation of the bladder", "causality_phrase": "following"}
2095
out:  {"cause": "Haloperidol", "effect": "NMS", "causality_phrase": "drug-related response to"}
2096
out:  {"cause": "etanercept for severe plaque psoriasis", "effect": "persistent hypoglycemia requiring the lowering and eventual elimination of his previous insulin regimen", "causality_phrase": "while treated with"}
2097
out:  {"cause": "ibuprofen", "effect": "meningitis", "causality_phrase": "induced"}
2098
out:  {"cause": "orally administered terbinafine", "effect": "Hepatobiliary disorders", "causality_phrase": "associated with"}
2099
out:  {"cause": "", "effect": "", "causality_

out:  {"cause": "ergotamine-induced arteriospasm", "effect": "arteriospasm", "causality_phrase": "by mechanical intra-arterial dilatation"}
2154
out:  {"cause": "initial infusion of rituximab", "effect": "acute coronary syndromes", "causality_phrase": "associated with"}
2155
out:  {"cause": "a high dose combined with a short infusion time", "effect": "anaphlactoid reactions", "causality_phrase": "increases the risk of"}
2156
out:  {"cause": "ferrous sulfate", "effect": "hypothyroid", "causality_phrase": "while taking"}
2157
out:  {"cause": "valproic acid-induced", "effect": "dementia", "causality_phrase": "Reversible"}
2158
out:  {"cause": "mannitol infusion", "effect": "ARF", "causality_phrase": "associated with"}
2159
out:  {"cause": "methotrexate therapy", "effect": "aseptic pleuropericarditis", "causality_phrase": "under"}
2160
out:  {"cause": "Methyldopa", "effect": "liver injury", "causality_phrase": "-induced"}
2161
out:  {"cause": "carbamazepine", "effect": "Diabetes Mellitus",

out:  {"cause": "foscarnet", "effect": "Electrolyte disorders", "causality_phrase": "associated with"}
2217
out:  {"cause": "quinapril therapy", "effect": "a dry, irritating cough", "causality_phrase": "within one month of starting"}
2218
out:  {"cause": "ifosfamide, occurring in a patient having received previous cisplatin chemotherapy and with poor kidney perfusion due to transient hypotension", "effect": "lethal anuria", "causality_phrase": "mainly due to"}
2219
out:  {"cause": "isoniazid", "effect": "pellagra", "causality_phrase": "may induce"}
2220
out:  {"cause": "cisplatin", "effect": "C. difficile colitis", "causality_phrase": "may cause"}
2221
out:  {"cause": "amphotericin B overdose", "effect": "secondary cardiac complications", "causality_phrase": "with"}
2222
out:  {"cause": "cimetidine", "effect": "hepatitis", "causality_phrase": "induced"}
2223
out:  {"cause": "omeprazole therapy", "effect": "Esophageal candidiasis", "causality_phrase": "receiving"}
2224
out:  {"cause": "

In [11]:
len(filtered_rows)

2271

In [12]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

0

In [13]:
filtered_rows['cause'] = cause_list
filtered_rows['effect'] = effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",5-fluorouracil,Squamous-cell carcinoma arising in a basal-cel...,treated with
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,receiving
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol,ARF,induced
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",chlorambucil was discontinued,clinical AML,the patient developed
...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",fludarabine phosphate,respiratory syncytial virus (RSV) infection,treated with
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...",a second course of flurbiprofen,biopsy-proven acute tubulointerstitial nephritis,associated with
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,male gynaecomastia,treated with


In [14]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [15]:
#get accuracy
from fuzzywuzzy import fuzz

def get_accuracy(filtered_rows):
    prediction= []
    e_prediction=[]
    c_prediction=[]
    for i in range(len(filtered_rows)):
        #print(i)
        cause_pred = filtered_rows['cause'][i]
        cause_t = filtered_rows['cause_t'][i]
        
        effect_pred = filtered_rows['effect'][i]
        effect_t = filtered_rows['effect_t'][i]

        # Slide through input_sentence with a window size of 8
        print('effect_pred',  effect_pred)
        print('effect_t',  effect_t)
        print('cause_t',  cause_t)
        print('cause_pred',  cause_pred)
        #if  isinstance(cause_pred, list):
            #prediction.append(0)
            #continue;


        score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
        #score = fuzz.ratio(cause_pred, cause_t)################medcause
        if score >90:################90 non -medcause
            pred_c = 1
            c_prediction.append(pred_c)
        else:
            pred_c = 0
            c_prediction.append(pred_c)
            
        score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
        #score = fuzz.ratio(effect_pred, effect_t)################medcause

        if score >90: ################90 non -medcause
            pred_e = 1
            e_prediction.append(pred_e)
        else:
            pred_e = 0
            e_prediction.append(pred_e)

        
        if ((pred_e ==1) & (pred_c ==  1)):
            prediction.append(1)
        else:
            prediction.append(0)
            
    

    return prediction, c_prediction, e_prediction


In [16]:
prediction, c_prediction, e_prediction = get_accuracy(filtered_rows)

effect_pred Squamous-cell carcinoma arising in a basal-cell epithelioma
effect_t Squamous-cell carcinoma
cause_t 5-fluorouracil
cause_pred 5-fluorouracil
effect_pred acute renal toxicity
effect_t acute renal toxicity
cause_t 5-fluorouracil
cause_pred long-term therapy with mitomycin C and 5-fluorouracil
effect_pred contact dermatitis
effect_t contact dermatitis
cause_t budesonide
cause_pred budesonide
effect_pred ARF
effect_t ARF
cause_t mannitol
cause_pred mannitol
effect_pred clinical AML
effect_t AML
cause_t chlorambucil
cause_pred chlorambucil was discontinued
effect_pred seizures and altered sensorium
effect_t altered sensorium
cause_t L-asparaginase
cause_pred L-asparaginase therapy
effect_pred hepatopathy
effect_t hepatopathy
cause_t carbamazepine
cause_pred a regimen of carbamazepine, lynestrenol and sodium valproate
effect_pred Diagnosis of sclerosing glomerulonephritis
effect_t sclerosing glomerulonephritis
cause_t anastrozole
cause_pred anastrozole use
effect_pred tracheal c

effect_pred lipoatrophy
effect_t lipoatrophy
cause_t Insulin
cause_pred Insulin
effect_pred SIADH
effect_t SIADH
cause_t lorcainide
cause_pred lorcainide therapy
effect_pred chest pain, elevated cardiac biomarkers, and acute left ventricular dysfunction
effect_t acute left ventricular dysfunction
cause_t methylphenidate
cause_pred a single dose of methylphenidate
effect_pred simultaneous transitional cell carcinoma of the renal pelvis and distal ureter without transitional cell carcinoma of the bladder
effect_t distal ureter without transitional cell carcinoma of the bladder
cause_t cyclophosphamide
cause_pred chronic cyclophosphamide therapy for nonHodgkin's lymphoma
effect_pred doubling of the lamotrigine blood level with symptoms of toxicity
effect_t doubling of the lamotrigine blood level
cause_t desmethylsertraline
cause_pred a total daily dose of 25 mg sertraline, with nondetectable sertraline and desmethylsertraline blood levels
effect_pred Massive pulmonary embolism
effect_t Ma

In [17]:
sum(prediction)/len(filtered_rows)

0.8370761778952004

In [18]:
sum(c_prediction)/len(filtered_rows)

0.9326287978863936

In [19]:
sum(e_prediction)/len(filtered_rows)

0.8811096433289299

In [20]:
filtered_rows['predictions'] = prediction

In [21]:
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase,predictions
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",5-fluorouracil,Squamous-cell carcinoma arising in a basal-cel...,treated with,1
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,receiving,1
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",budesonide,contact dermatitis,causing,1
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol,ARF,induced,1
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",chlorambucil was discontinued,clinical AML,the patient developed,1
...,...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",fludarabine phosphate,respiratory syncytial virus (RSV) infection,treated with,1
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,severe myelosuppression,due to,0
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...",a second course of flurbiprofen,biopsy-proven acute tubulointerstitial nephritis,associated with,1
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,male gynaecomastia,treated with,1


In [22]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#filtered_rows.to_csv('data/out/RAG_causality/CE_extracts/randomshotCOMB_ADE_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/task2/gemma/rand_ADE_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [23]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
